In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numpy pandas matplotlib gym stable-baselines3 torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 64.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [ ]:
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO

In [ ]:
import pandas as pd
fp='/content/drive/MyDrive/study_planner_dataset.csv'
df=pd.read_csv(fp)
df.head()

,User_ID,Subject,Priority,Past_Score,Improvement,Course_Size,Days_to_Test,Study_Time
0,1,Math,1,71,2.82,9,31,3.3
1,1,Science,5,45,8.76,7,34,2.0
2,1,English,4,75,5.76,14,58,2.2
3,1,History,5,72,8.24,8,6,1.8
4,1,Science,1,51,-2.50,7,26,1.2


# **RL Environment**

In [ ]:
import datetime
class StudyPlannerEnv(gym.Env):
  def __init__(self, data):
    super(StudyPlannerEnv,self).__init__()

    self.data=data
    self.current_step=0

    self.observation_space=spaces.Box(low=0,high=100, shape=(5,), dtype=np.float32)
    self.action_space=spaces.Discrete(6)

  def update_user_inpt(self, user_id, subject, np=None, ntd=None):
    index=self.data[(self.data['User_ID']==user_id) & (self.data['Subject']==subject)].index

    if np is not None:
      self.data.loc[index, 'Priority']=np
    if ntd is not None:
      currdate=datetime.today()
      days_to_test=(ntd-currdate).days
      self.data.loc[index,'Days_to_Test']=max(0, days_to_test)

  def reset(self):
    self.current_step = 0
    state = self.data.iloc[self.current_step, 2:7].values  # Ensure exactly 5 features
    return state.astype(np.float32)

  def step(self, action):
    self.current_step += 1

    if self.current_step >= len(self.data):
        done = True
        return np.zeros(5, dtype=np.float32), 0, done, {}

    next_state = self.data.iloc[self.current_step, 2:7].values.astype(np.float32)  # Ensure correct shape

    improvement = self.data.iloc[self.current_step]["Improvement"]
    reward = improvement if improvement > 0 else -1

    done = False
    return next_state, reward, done, {}


In [ ]:
!pip install shimmy>=2.0

In [ ]:
from stable_baselines3.common.env_util import make_vec_env
env=make_vec_env(lambda: StudyPlannerEnv(df), n_envs=1)
model=PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)
model.save('ppo_study_planner')

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gym/core.py:256: DeprecationWarning: WA

Using cuda device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | 685      |
| time/              |          |
|    fps             | 429      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 200        |
|    ep_rew_mean          | 685        |
| time/                   |            |
|    fps                  | 441        |
|    iterations           | 2          |
|    time_elapsed         | 9          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01148056 |
|    clip_fraction        | 0.072      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.78      |
|    explained_variance   | -0.000886  |
|    learning_rate        |

In [ ]:
model=PPO.load('ppo_study_planner')
env=StudyPlannerEnv(df)
obs=env.reset()
for _ in range(10):
  action,_states=model.predict(obs)
  obs,rew,done,_=env.step(action)
  print("Suggested study time: ", action, "Reward: ", rew)
  if done:
    break

Suggested study time:  5 Reward:  8.76
Suggested study time:  0 Reward:  5.76
Suggested study time:  3 Reward:  8.24
Suggested study time:  1 Reward:  -1
Suggested study time:  0 Reward:  4.88
Suggested study time:  2 Reward:  4.52
Suggested study time:  5 Reward:  1.29
Suggested study time:  4 Reward:  -1
Suggested study time:  3 Reward:  -1
Suggested study time:  3 Reward:  9.46


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
